# Car Price Prediction Model Building

In this notebook, I build a powerful price prediction model using Random Forest Regressor and other techniques, who ables to predict closest prices of cars on unseen data. Outline:

- Dataset Overview:
- Data Preprocessing:
- Data Binning:
- Model Building:
random forest 
mean_absolute_error
coorelation
Transformers
pipeline
training another model for Predicting 5 years price

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import pickle

# Load and prepare the dataset

In [2]:
df = pd.read_csv("OLX_cars_dataset00.csv")


# Datset Overview

In [3]:
df.head(20)

,Ad ID,Car Name,Make,Model,Year,KM's driven,Price,Fuel,Registration city,Car documents,Assembly,Transmission,Condition,Seller Location,Description,Car Features,Images URL's,Car Profile
0,1079071571,fresh import Passo 2021model,Toyota,Passo,2021,54000,4190000,Petrol,Unregistered,Original,Imported,Automatic,Used,"Airline Avenue, Islamabad","it's 2021 model fresh import, perfect engine s...","ABS, Air Bags, AM/FM Radio, CD Player, Cassett...",['https://images.olx.com.pk/thumbnails/4039460...,https://www.olx.com.pk/item/fresh-import-passo...
1,1080125520,Suzuki ravi,Suzuki,Ravi,2018,95000,1300000,Petrol,Karachi,Original,Local,Manual,Used,"Kahuta, Rawalpindi",Suzuki ravi 2018 col,AM/FM Radio,['https://images.olx.com.pk/thumbnails/4102504...,https://www.olx.com.pk/item/suzuki-ravi-iid-10...
2,1080748789,Suzuki bolan 2015 contact 03112271054,Suzuki,Bolan,2015,50000,800000,Petrol,Karachi,Original,Local,Manual,Used,"Lyari Expressway, Karachi",Suzuki bolan model 2015 reg 2022 ghadi me koi ...,Rear speakers,['https://images.olx.com.pk/thumbnails/4139520...,https://www.olx.com.pk/item/suzuki-bolan-2015-...
3,1076081635,Diahatsu Move 2013,Daihatsu,Move,2013,94000,2155000,Petrol,Lahore,Original,Imported,Automatic,Used,"New Amir Town, Lahore",Move Push start \nHome Used car. \nModel 2013....,"ABS, Air Bags, Air Conditioning, Alloy Rims, A...",['https://images.olx.com.pk/thumbnails/3865337...,https://www.olx.com.pk/item/diahatsu-move-2013...
4,1080812928,Suzuki Swift DLX 2011 miner tuchap,Suzuki,Swift,2011,126544,1440000,Petrol,Karachi,Original,Local,Manual,Used,"Shadman 2, Karachi",Suzuki Swift DLX\nAbS Break 100%\nengine 100%\...,"ABS, Air Conditioning, Alloy Rims, AM/FM Radio...",['https://images.olx.com.pk/thumbnails/4143509...,https://www.olx.com.pk/item/suzuki-swift-dlx-2...
5,1079496823,WagonR AGS total genuine,Suzuki,Wagon R,2020,54000,2830000,Petrol,Lahore,Original,Local,Automatic,Used,"Abbot Road, Sialkot",wagon r total genuine AGS new tyre,"ABS, Air Bags, Alloy Rims, AM/FM Radio, CD Pla...",['https://images.olx.com.pk/thumbnails/4065083...,https://www.olx.com.pk/item/wagonr-ags-total-g...
6,1080333426,Daihatsu Mira,Daihatsu,Mira,2013,140000,2150000,Petrol,Karachi,Original,Imported,Automatic,Used,"North Nazimabad - Block A, Karachi",bilkul okay condition me hen engine change hen...,"ABS, Air Bags, Air Conditioning, AM/FM Radio, ...",['https://images.olx.com.pk/thumbnails/4114961...,https://www.olx.com.pk/item/daihatsu-mira-iid-...
7,1080784758,Suzuki Bolan 2016 genuine condition low mileage,Suzuki,Bolan,2016,32000,1050000,Petrol,Karachi,Original,Local,Manual,Used,"Korangi Industrial Area, Karachi",SUZUKI BOLANSuzuki Bolan 2016 genuine conditio...,"Alloy Rims, AM/FM Radio",['https://images.olx.com.pk/thumbnails/4141723...,https://www.olx.com.pk/item/suzuki-bolan-2016-...
8,1076448625,2004 Honda City,Honda,City IDSI,2004,160000,1230000,Petrol,Sindh,Original,Local,Manual,Used,"Allama Iqbal Town - Huma Block, Lahore",A very well maintained - Good consumption\nno...,"ABS, Air Conditioning, Alloy Rims, CD Player, ...",['https://images.olx.com.pk/thumbnails/3886393...,https://www.olx.com.pk/item/2004-honda-city-ii...
9,1080820574,car Bashir s 80 janwan Hain 03218823600,Suzuki,Cultus VXR,2008,75000,950000,Petrol,Faisalabad,Original,Local,Manual,Used,"Al Najaf Colony, Faisalabad",Ander s hotel janvan Hain by Matrek avlebal Ha...,"Air Conditioning, AM/FM Radio, CD Player, Cool...",['https://images.olx.com.pk/thumbnails/4143955...,https://www.olx.com.pk/item/car-bashir-s-80-ja...



# 1. Data Cleaning and Preprocessing

In [4]:

print(f"Duplicated Rows: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
df.drop(columns=["Ad ID", "Car Name", "Condition", "Seller Location", "Registration city", 
                 "Description", "Car Features", "Images URL's", "Car Profile"], inplace=True)


Duplicated Rows: 201


# Remove unwanted car models and future year

In [5]:

df = df[(df["Model"] != "Civic VTi") & (df["Model"] != "Civic EXi") & (df["Model"] != "Civic VTi Oriel") &
        (df["Model"] != "Cervo") & (df["Model"] != "Every Wagon") & (df["Model"] != "Liana") &
        (df["Model"] != "Mehran VX") & (df["Model"] != "Khyber") & (df["Model"] != "Cultus VXL") &
        (df["Model"] != "Corolla Assista") & (df["Model"] != "Corolla Axio") & (df["Model"] != "Surf") &
        (df["Model"] != "Prius") & (df["Model"] != "ISIS")]
df = df[df["Year"] != 2024]

In [6]:
df.drop_duplicates(inplace=True)

# Add 'Age' of the vehicle and create 'Year_Range' and 'KM's driven_Range' bins

In [7]:

df['Age'] = 2024 - df['Year']
bins_year = [1999, 2004, 2008, 2012, 2016, 2020, 2024]
labels_year = [1, 2, 3, 4, 5, 6]
df["Year_Range"] = pd.cut(df["Year"], bins=bins_year, labels=labels_year).astype("int32")


In [8]:
bins_km = [0, 30000, 60000, 90000, 120000, 150000, 180000, 210000, 224000, 227000, 300000, 
           330000, 360000, 390000, 410000, 440000, 470000, 500000, 533530]
labels_km = list(range(1, len(bins_km)))
df["KM's driven_Range"] = pd.cut(df["KM's driven"], bins=bins_km, labels=labels_km).astype("int32")


# 3. Splitting Data for Training the First Model

In [9]:

X1 = df.drop("Price", axis=1)
y1 = df["Price"]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.15, random_state=42)


In [10]:
X1_train

,Make,Model,Year,KM's driven,Fuel,Car documents,Assembly,Transmission,Age,Year_Range,KM's driven_Range
5562,Suzuki,Swift,2013,102785,Petrol,Original,Local,Manual,11,4,4
6513,Honda,City IVTEC,2013,101500,Petrol,Original,Local,Automatic,11,4,4
9000,Toyota,Corolla XLI,2007,200000,Petrol,Original,Local,Manual,17,2,7
8113,Honda,City IDSI,2007,180000,Petrol,Original,Local,Manual,17,2,6
2221,Toyota,Corrolla Altis,2008,55000,Petrol,Original,Imported,Automatic,16,2,2
...,...,...,...,...,...,...,...,...,...,...,...
5918,Suzuki,Bolan,2012,27000,Petrol,Original,Local,Manual,12,3,1
5347,Honda,Civic Prosmetic,2004,205666,Petrol,Original,Local,Automatic,20,1,7
5557,Honda,Civic Oriel,2013,71000,Petrol,Original,Local,Automatic,11,4,3
863,Hyundai,Santro,2004,90000,Petrol,Original,Imported,Manual,20,1,3


In [11]:
ct1 = ColumnTransformer(transformers=[
    ("oneHotEncoder", OneHotEncoder(sparse=False, drop="first"), [0,1,4,5,6,7])
], remainder="passthrough")

ct2 = ColumnTransformer(transformers=[
    ("minMaxScaler", MinMaxScaler(), [2,3,8,9])
], remainder="passthrough")

ct3 = ColumnTransformer(transformers=[
    ("quantileTransformers", QuantileTransformer(output_distribution="normal"), slice(0, None))
])

ct4 = RandomForestRegressor()


In [12]:
model1= Pipeline([
    ("first", ct1),
    ("second", ct2),
    ("third", ct3),
    ("fourth", ct4)
])

# Train the first model

In [13]:

model1.fit(X1_train, y1_train)


Pipeline(steps=[('first',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oneHotEncoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  [0, 1, 4, 5, 6, 7])])),
                ('second',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('minMaxScaler',
                                                  MinMaxScaler(),
                                                  [2, 3, 8, 9])])),
                ('third',
                 ColumnTransformer(transformers=[('quantileTransformers',
                                                  QuantileTransformer(output_distribution='normal'),
                                                  slice(0, None, None))])),
                ('fourth', RandomForestRegressor())]

# Save the trained model

In [14]:

with open('final_prediction_current_price.pkl', 'wb') as file:
    pickle.dump(model1, file)


In [15]:
# Model Prediction for First Model
y1_pred = model1.predict(X1_test)
print(f"First Model RMSE: {np.sqrt(mean_squared_error(y1_test, y1_pred))}")


First Model RMSE: 272169.24579528125


# 4. Training the Second Model for Future Price Predictions
# Adding Future Price Columns

In [17]:

inflation_rate = 0.08
tax_rate = 0.01
years = [1, 2, 3, 4, 5]
for year in years:
    df[f'Price_{year}_Year'] = df['Price'] * ((1 + inflation_rate + tax_rate) ** year)


In [18]:
X2 = df.drop(columns=[f'Price_{year}_Year' for year in years] + ['Price'])
y2 = df[[f'Price_{ye
inflation_rate = 0.08
tax_rate = 0.01
years = [1, 2, 3, 4, 5]
for year in years:
    df[f'Price_{year}_Year'] = df['Price'] * ((1 + inflation_rate + tax_rate) ** year)
ar}_Year' for year in years]]


In [19]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)


In [20]:
# Pipeline for Second Model with MultiOutput Regressor
model2 = Pipeline([
    ('preprocessor', ct1),  # Apply preprocessing from first model for consistency
    ('regressor', MultiOutputRegressor(RandomForestRegressor(random_state=42)))
])

# Train the second model


In [22]:
model2.fit(X2_train, y2_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oneHotEncoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  [0, 1, 4, 5, 6, 7])])),
                ('regressor',
                 MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42)))])

# Save the second model


In [23]:
joblib.dump(model2, 'rought2future.pkl')
print("Second Model trained and saved successfully!")


Second Model trained and saved successfully!


In [24]:
y2_pred = model2.predict(X2_test)
print(f"Second Model RMSE: {np.sqrt(mean_squared_error(y2_test, y2_pred))}")


Second Model RMSE: 367243.84347902733


# Output predicting 

In [2]:
import pickle
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import pickle 
def load_and_predict_price(data, model_file='final_prediction_current_price.pkl'):
    with open(model_file, 'rb') as file:
        loaded_model = pickle.load(file)
    if isinstance(data, dict):
        data = pd.DataFrame([data])
    elif isinstance(data, list):
        data = pd.DataFrame(data, columns=loaded_model.named_steps['first'].get_feature_names_out())
    return loaded_model.predict(data)[0]

def load_and_predict_future_prices(data, model_file='multioutput_car_price_second_model.pkl'):
    model = joblib.load(model_file)
    if isinstance(data, dict):
        data = pd.DataFrame([data])
    return model.predict(data)[0]
sample_input = {
    "Make": "Suzuki",
    "Model": "Swift",
    "Year": 2013,
    "KM's driven": 102785,
    "Fuel": "Petrol",
    "Car documents": "Original",
    "Assembly": "Local",
    "Transmission": "Manual",
    "Age": "11",
    "Year_Range": 4, 
    "KM's driven_Range": 4
}

# Predict current price
predicted_price = load_and_predict_price(sample_input)
print(f"Predicted Current Price: {predicted_price}")

# Predict future prices
predicted_future_prices = load_and_predict_future_prices(sample_input)
print(f"Predicted Future Prices: {predicted_future_prices}")

Predicted Current Price: 1624475.0
Predicted Future Prices: [1736751.5        1897455.105      2068226.06445    2254366.4102505
 2459721.18550088]


# depreciation agr dikhanan ha to 

Enter the number of years into the future you want to predict the price for: 3

******The predicted price in 3 years will be approximately: 1392784.2531249998*****


# nechay user s hm na irrelevant hide kr dia ha taki confuse na ho wo feature
# change hoga according to the year and kms driver as we have creaetd bins etc

In [7]:
import pandas as pd
import pickle
import joblib
from ipywidgets import widgets, Button, VBox, Label, HTML
from IPython.display import display, clear_output

# Load models
def load_model(model_file):
    with open(model_file, 'rb') as file:
        return pickle.load(file)

def load_joblib_model(model_file):
    return joblib.load(model_file)

current_price_model = load_model('final_prediction_current_price.pkl')
future_price_model = load_joblib_model('multioutput_car_price_second_model.pkl')

# Load the dataset for dropdown options
car_data = pd.read_csv('OLX_cars_dataset00.csv')
unique_makes = car_data['Make'].dropna().unique().tolist()
unique_models = car_data['Model'].dropna().unique().tolist()

# Helper functions for derived features
def calculate_age(year):
    return 2024 - year

def calculate_year_range(year):
    bins_year = [1999, 2004, 2008, 2012, 2016, 2020, 2024]
    labels_year = [1, 2, 3, 4, 5, 6]
    return pd.cut([year], bins=bins_year, labels=labels_year).astype("int32")[0]

def calculate_km_range(km_driven):
    bins_km = [0, 30000, 60000, 90000, 120000, 150000, 180000, 210000, 224000, 227000, 300000, 
               330000, 360000, 390000, 410000, 440000, 470000, 500000, 533530]
    labels_km = list(range(1, len(bins_km)))
    return pd.cut([km_driven], bins=bins_km, labels=labels_km).astype("int32")[0]

# Prediction functions
def load_and_predict_price(data):
    if isinstance(data, dict):
        data = pd.DataFrame([data])
    return current_price_model.predict(data)[0]

def load_and_predict_future_prices(data):
    if isinstance(data, dict):
        data = pd.DataFrame([data])
    return future_price_model.predict(data)[0]

# Function to display the main input form
def display_input_form():
    clear_output()
    display(input_widgets)
    predict_button = Button(description="Predict Current Price")
    predict_button.on_click(lambda b: make_prediction(
        make_widget.value, model_widget.value, year_widget.value,
        km_driven_widget.value, fuel_widget.value,
        car_documents_widget.value, assembly_widget.value,
        transmission_widget.value
    ))
    display(predict_button)

# Function to handle predictions and display output
def make_prediction(make, model, year, km_driven, fuel, car_documents, assembly, transmission):
    # Prepare the input data with derived fields
    age = calculate_age(year)
    year_range = calculate_year_range(year)
    km_range = calculate_km_range(km_driven)
    sample_input = {
        "Make": make, "Model": model, "Year": year, "KM's driven": km_driven,
        "Fuel": fuel, "Car documents": car_documents, "Assembly": assembly,
        "Transmission": transmission, "Age": age, "Year_Range": year_range,
        "KM's driven_Range": km_range
    }
    
    # Predict current price
    predicted_price = load_and_predict_price(sample_input)
    clear_output()
    
    # Display predicted current price
    display(HTML(f"<div style='border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; margin: 10px 0; font-size: 1.2em;'>"
                 f"<strong>Predicted Current Price:</strong> PKR:{predicted_price:,.2f}</div>"))

    # Buttons to proceed or go back
    future_price_button = Button(description="Check Future Price")
    future_price_button.on_click(lambda b: check_future_price(predicted_price, sample_input))
    
    back_button = Button(description="Back to Input")
    back_button.on_click(lambda b: display_input_form())
    
    display(VBox([future_price_button, back_button]))

# Function to handle future price prediction
def check_future_price(predicted_price, sample_input):
    clear_output(wait=True)
    
    # Display current price
    display(HTML(f"<div style='border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; margin: 10px 0; font-size: 1.2em;'>"
                 f"<strong>Predicted Current Price:</strong> PKR:{predicted_price:,.2f}</div>"))
    
    # Dropdown to select years for future prediction
    year_dropdown = widgets.Dropdown(
        options=[1, 2, 3, 4, 5],
        description="Years Ahead"
    )
    
    # Function to predict future price
    def predict_future(b):
        years = year_dropdown.value
        future_prices = load_and_predict_future_prices(sample_input)
        future_price = future_prices[years - 1]
        clear_output(wait=True)
        
        # Display future price
        display(HTML(f"<div style='border: 2px solid #2196F3; padding: 10px; border-radius: 8px; margin: 10px 0; font-size: 1.2em;'>"
                     f"<strong>Predicted Future Price in {years} years:</strong> PKR:{future_price:,.2f}</div>"))
        
        # Back button to return to input form
        back_to_input_button = Button(description="Back to Input")
        back_to_input_button.on_click(lambda b: display_input_form())
        display(back_to_input_button)
    
    # Future prediction button
    future_button = Button(description="Predict Future Price")
    future_button.on_click(predict_future)
    
    # Back button to return to input form
    back_button = Button(description="Back to Input")
    back_button.on_click(lambda b: display_input_form())
    
    display(VBox([Label("Select years ahead for future price prediction:"), year_dropdown, future_button, back_button]))

# Creating interactive widgets for user input
make_widget = widgets.Dropdown(options=unique_makes, description="Company")
model_widget = widgets.Dropdown(options=unique_models, description="Model")
year_widget = widgets.IntSlider(description="Year", min=1999, max=2024, value=2013)
km_driven_widget = widgets.IntText(description="KM's Driven", value=102785)
fuel_widget = widgets.Dropdown(options=["Petrol", "Diesel", "CNG", "Hybrid"], description="Fuel", value="Petrol")
car_documents_widget = widgets.Dropdown(options=["Original", "Duplicate"], description="Car Documents", value="Original")
assembly_widget = widgets.Dropdown(options=["Local", "Imported"], description="Assembly", value="Local")
transmission_widget = widgets.Dropdown(options=["Manual", "Automatic"], description="Transmission", value="Manual")

# Main input form widget group
input_widgets = VBox([make_widget, model_widget, year_widget, km_driven_widget, fuel_widget,
                      car_documents_widget, assembly_widget, transmission_widget])

# Display the initial input form
display_input_form()


HTML(value="<div style='border: 2px solid #2196F3; padding: 10px; border-radius: 8px; margin: 10px 0; font-siz…

Button(description='Back to Input', style=ButtonStyle())